In [15]:
import $file.hw6stdlib
import hw6stdlib._

Compiling hw6stdlib.sc


import $file.$        

import hw6stdlib._

# Recitation 6

In this recitation we will write a well-formedness checker for Lettuce. Recall that well-formedness as we have defined it is a property of programs in which variables are only used after they have been defined.

For example:

```
//exNotWF below
let x = 5 in
  y + x
```

is not a well-formed lettuce program because `y` is not defined when it is used.

```
//exWF below
let y = 10 in
  let x = 5 in 
    y + x
```

in this example `y` _is_ defined, as is `x`, when used on the last line of the program. Therefore, this is an example of a well-formed Lettuce expression.




Here we give the abstract syntax of Lettuce expressions:

$$\begin{array}{rcll}
\mathbf{Expr} & ::= & Num(\mathbb{Z}) \\
 & | & Bin(\mathbb{B}) \\
 & | & Ident(\mathbf{String}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Minus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Pow (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Neg (\mathbf{Expr}) \\
 & | & Eq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & And ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & Or ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) \\
 & | & Let( \mathbf{String}, \mathbf{Expr}, \mathbf{Expr}) \\
\end{array}$$

and it's encoding in Scala:

In [16]:
sealed trait Expr
case class Num(x : Integer) extends Expr
case class Bin(x : Bool) extends Expr
case class Ident(x : String) extends Expr
case class Plus(x : Expr, y : Expr) extends Expr
case class Minus(x : Expr, y : Expr) extends Expr
case class Mult(x : Expr, y : Expr) extends Expr
case class Pow(x : Expr, y : Expr) extends Expr
case class Neg(x : Expr) extends Expr
case class Eq(x : Expr, y : Expr) extends Expr
case class And(x : Expr, y : Expr) extends Expr
case class Or(x : Expr, y : Expr) extends Expr
case class IfThenElse(p : Expr, t : Expr, f : Expr) extends Expr
case class Let(id : String, x : Expr, y : Expr) extends Expr

defined trait Expr
defined class Num
defined class Bin
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Pow
defined class Neg
defined class Eq
defined class And
defined class Or
defined class IfThenElse
defined class Let

Below we have encoded the two examples from above:

In [17]:
val exNotWF = Let("x", Num(Positive(five)), Plus(Ident("x"), Ident("y")))

val exWF = Let("y", Num(Positive(ten)), exNotWF)

exNotWF: Let = Let(x,Num(Positive(Succ(Succ(Succ(Succ(Succ(Zero))))))),Plus(Ident(x),Ident(y)))
exWF: Let = Let(y,Num(Positive(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Zero)))))))))))),Let(x,Num(Positive(Succ(Succ(Succ(Succ(Succ(Zero))))))),Plus(Ident(x),Ident(y))))

## Exercise: Well Formedness Checker for Lettuce

Now define the well-formedness checker for Lettuce. The isIn function below, which checks if a given string is in a list of strings, may prove to be helpful.

In [18]:
def isIn(xs : List[String], x : String) : Bool = fold(( (y : String, acc : Bool) => or(acc, string_eq(x,y))), False, xs)

defined function isIn

In [19]:
def isWellFormed(expr : Expr, context : List[String]) : Bool = expr match {
    case Num(x)    => True
    case Bin(x)    => True
    case Ident(id) => isIn(context, id)
    case Plus(x,y) => and(isWellFormed(x, context), isWellFormed(y, context))
    case Minus(x,y) => and(isWellFormed(x, context), isWellFormed(y, context))
    case Mult(x,y) => and(isWellFormed(x, context), isWellFormed(y, context))
    case Pow(x,y) => and(isWellFormed(x, context), isWellFormed(y, context))
    case Neg(p) => isWellFormed(p, context)
    case Eq(x,y) => and(isWellFormed(x, context), isWellFormed(y, context))
    case And(x,y) => and(isWellFormed(x, context), isWellFormed(y, context))
    case Or(x,y) => and(isWellFormed(x, context), isWellFormed(y, context))
    case IfThenElse(p,x,y) => and(isWellFormed(p, context), and(isWellFormed(x, context), isWellFormed(y, context)))
    case Let(id,e1,e2) => and(isWellFormed(e1, context), isWellFormed(e2, Cons(id, context)))
}

defined function isWellFormed

In [20]:
assert(isWellFormed(Plus(Num(Positive(one)), Num(Negative(one))), Empty) == True)
assert(isWellFormed(exNotWF, Empty) == False)
assert(isWellFormed(exWF, Empty) == True)

## Exercise: Write rules from code
Given the code for Nat plus (coppied bellow) write the rules that are equivalent to the code using this format on paper:

$ \begin{array}{c}
???\ \ \ \ ???\ \ \ \ ???\\
\hline
\text{nat_plus}(???, ???)\ \text{returns}\ ??? \\
\end{array} \mathbf{(Rule\ Name)} $

In [21]:
def nat_plus(x : Nat, y : Nat) : Nat = x match {
    case Zero         => y
    case Succ(x_pred) => Succ(nat_plus(x_pred, y))
}

defined function nat_plus

## Solution
$ \begin{array}{c}
\\
\hline
\text{nat_plus}(\text{Zero}, y)\ \text{returns}\ y \\
\end{array} \mathbf{(Plus\ Zero)} $

$ \begin{array}{c}
\text{nat_plus}(x_\text{pred}, y)\ \text{returns}\ n\\
\hline
\text{nat_plus}(\text{Succ}(x_\text{pred}, y)\ \text{returns}\ \text{Succ}(n) \\
\end{array} \mathbf{(Plus\ Succ)} $